In [1]:
%pip install openpyxl
import pandas as pd
df_articles = pd.read_csv('../data/GVFC_extension_multimodal.csv')
df_headlines = pd.read_excel('../data/GVFC_headlines_and_annotations.xlsx')

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(df_articles.shape, df_articles.columns.tolist())
print(df_headlines.shape, df_headlines.columns.tolist())

(1300, 17) ['id', 'article_url', 'headline', 'google_visual_api_web_entities_detection_on_lead_image', 'automatically_generated_caption_of_lead_image', 'lead_3_sentences_summary_of_full_article_text', 'presum_summary_of_full_article_text', 'Q1 Relevant', 'Q2 Focus', 'Q3 Theme1', 'Q3 Theme2', 'V1image', 'V2ethnicity', 'V3relevance', 'V4relevance', 'Q4 Image1', 'Q4 Image2']
(2990, 5) ['news_title', 'Q1 Relevant', 'Q2 Focus', 'Q3 Theme1', 'Q3 Theme2']


In [3]:
df_articles.isna().mean().sort_values(ascending=False).head(10)


google_visual_api_web_entities_detection_on_lead_image    0.001538
V4relevance                                               0.001538
lead_3_sentences_summary_of_full_article_text             0.000769
presum_summary_of_full_article_text                       0.000769
id                                                        0.000000
Q3 Theme2                                                 0.000000
Q4 Image1                                                 0.000000
V3relevance                                               0.000000
V2ethnicity                                               0.000000
V1image                                                   0.000000
dtype: float64

In [4]:
df_articles['article_summary'] = df_articles['presum_summary_of_full_article_text']

In [5]:
KEEP_COLS = ['id',
             'article_url',
             'headline',
             'article_summary',      # we just created this alias
             'Q2 Focus',             # optional: perpetrator / victim / policy …
             'Q3 Theme1', 'Q3 Theme2'  # optional for later slicing
            ]

df = df_articles[KEEP_COLS].copy()
print(df.shape)

(1300, 7)


In [6]:
df = df.rename(columns={
    'headline':       'headline_text',
    'article_summary':'body_text',
    'article_url':    'url'
})

df = df.dropna(subset=['headline_text', 'body_text'])
print(f"Rows after dropping empties: {len(df)}")

Rows after dropping empties: 1299


In [7]:
for col in ['headline_text', 'body_text']:
    df[col] = (df[col]
                 .astype(str)              # in case of non‑string NaNs
                 .str.replace(r'\s+', ' ', regex=True)
                 .str.strip())

In [8]:
sample = df.sample(3, random_state=1)
for _, row in sample.iterrows():
    print("—" * 80)
    print("HEADLINE:", row['headline_text'])
    print("BODY    :", row['body_text'][:140], "…")

————————————————————————————————————————————————————————————————————————————————
HEADLINE: Twitter Users Taunt NRA With 'Thoughts And Prayers'Amid Reports Of Financial Struggles
BODY    : the national rifle association has ditched free coffee and yanked water coolers at its virginia headquarters due to a cash crunch. it comes  …
————————————————————————————————————————————————————————————————————————————————
HEADLINE: "We are a better city": Emotional mayor decries Chicago gun violence
BODY    : chicago's mayor calls for increased accountability for gun assailants. at least 11 people were killed and 70 wounded in a spate of shootings …
————————————————————————————————————————————————————————————————————————————————
HEADLINE: Pittsburgh shooting suspect said he wanted all Jews to die: police
BODY    : robert gregory bowers is charged with 11 counts of aggravated assault and 13 counts of ethnic intimidation. president trump donald john trum …


In [9]:
df = df.reset_index(drop=True)
CLEAN_PATH = "../data/clean_gvfc_sentiment.csv"
df.to_csv(CLEAN_PATH, index=False)
print("Clean table saved to", CLEAN_PATH)

Clean table saved to ../data/clean_gvfc_sentiment.csv


In [10]:
print("Average headline length:", df['headline_text'].str.len().mean())
print("Average body length    :", df['body_text'].str.len().mean())

Average headline length: 76.44341801385681
Average body length    : 423.891454965358
